File: cash_flow.ipynb\
Author: alexkobz\
Date: 19-11-2024\
Task: DSRFU-277

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = pd.to_datetime(last_day_month)
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF, RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


#### Шаг 1. Загрузка stage таблиц 

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [4]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (101518, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,NaN,NaN,NaN,None,2.0,2.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,5035,None,None,"""Акционерная энергетическая компания ""Комиэнер...",None,RU0006753647,Комиэнерго-1-ап,NaN,2-01-00053-A,2003-07-01T00:00:00,ФКЦБ,NaN,None,None,None,None,Акция,None,None,NaN,0.0,NaN,None,None,NaN,RU,NaN,RUB,1.000,NaN,NaN,NaN,Погашен,2.200000e+07,2.200000e+07,NaN,NaN,None,None,None,NaN,None,None,None,Комиэнерго Фл,00107063,Электроэнергетика,84326.0,7802312751,RU,Республика Коми,None,NaN,None,None,NaN,None,2008-04-08T00:00:00,0.0,None,Привилег-ая,Осн-ой,None,2023-08-25T18:55:08,NaN,Нет информации,NaN,None,NaN,Исключен с РТС с 03.04.2008г.,NaN,None,None,None,NaN,NaN,None,None,NaN,None,False,None,RU,1000,1000,NaN,None,NaN,False,False,None,NaN,NaN,None,None,101518,601
1,2.0,2.0,NaN,NaN,NaN,None,2.0,2.0,2.0,2.0,2.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,N

In [5]:
Calendar = RuDataDF("CalendarV2").df
Calendar.head()

CalendarV2 started
CalendarV2 finished. CalendarV2 shape (885042, 81)


,action_id,fintool_rusbonds_type,issuer_rusbonds_type,original_country,update_date,securityType,privateDist,country,listing_level,type_convertation,literid_typeoperation,act_code,status,status_date,literid_status,pay_decision,unpayed_decision_reason,payed_details,fininstId,issuerName,id,isiNcode,regCode,nickname,nrDcode,finToolID,type,eventID,typeOperation,rateDate,beginPeriod,endPeriod,couponPeriod,couponRate,pay1Bond,moneyFlowVal,eventTotalVol,daysFromDist,execution,payDate,fixDate,mtyDate,mtyPart,offerDate,buyBackPrice,begOrder,endOrder,eventPeriod,convert2FinToolID,coefficient,beginConvDate,endConvDate,defaultID,defaultType,typeEvent,defaultDate,realPayDate,note,eventType,eventDate,beginEventPer,value,faceFTName,pay1Security,moneyFlowPlan,protocolDate,pay1Bond_frac,currencyFTName,paymentCurrencyFTName,revisedPay1Bond,recomendPay1Security,recomendRateDate,recomendFixDate,recomendPayDate,recomendPayDateNominee,fintoolType,dividend_currency,ex_dividend_date,declared_pay_date_nominee,counter,rn
0,604078,C,None,US,2023-09-04T15:09:34,обыкновенная акция,False,US,NaN,None,1099.0,DVCA,None,None,NaN,Выплачивать деньгами,None,None,98662.0,McKesson Corporation,US58155Q1031,US58155Q1031,US58155Q1031,McKesson Corporation-ао,US58155Q1031,114130,None,215.0,EM,1999-01-27T00:00:00,2000-01-01T00:00:00,2000-08-31T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0,None,1999-03-01T00:00:00,None,NaN,None,NaN,None,None,244.0,None,NaN,None,None,NaN,None,None,None,None,None,DIV,1999-04-01T00:00:00,2000-01-01T00:00:00,NaN,USD,0.06000,NaN,1999-02-01T00:00:00,0.0,None,None,NaN,0.06000,1999-01-27T00:00:00,None,None,None,Акция,USD,1999-02-25T00:00:00,None,885042,4001
1,604748,C,None,NL,2023-09-04T15:09:47,обыкновенная акция,False,CW,NaN,None,1099.0,DVCA,None,None,NaN,Выплачивать деньгами,None,None,101934.0,SLB,AN8068571086,AN8068571086,AN8068571086,SLB-ао,AN8068571086,126623,None,215.0,EM,1999-01-20T00:00:00,2000-01-01T00:00:00,2000-08-31T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0,None,1999-02-24T00:00:00,None,NaN,None,NaN,None,None,244.0,None,NaN,None,None,NaN,None,None,None,None,None,DIV,1999-04-02T00:00:00,2000-01-01T00:00:00,NaN,USD,0.09375,NaN,1999-01-25T00:00:00,0.0,None,None,NaN,0.09375,1999-01-20T00:00:00,None,None,None,Акция,USD,1999-02-22T00:00:00,None,885042,4002
2,604233,C,None,US,2023-09-04T15:09:40,обыкновенная акция,False,US,NaN,None,1099.0,DVCA,None,None,NaN,Выплачивать деньгами,None,None,100287.0,Kimberly-Clark Corp,US4943681035,US4943681035,US4943681035,Kimberly-Clark Corp-ао,US4943681035,121824,None,215.0,EM,1999-02-25T00:00:00,2000-01-01T00:00:00,2000-08-31T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0,None,1999-03-05T00:00:00,None,NaN,None,NaN,None,None,244.0,None,NaN,None,None,NaN,None,None,None,None,None,DIV,1999-04-02T00:00:00,2000-01-01T00:00:00,NaN,USD,0.26000,NaN,1999-03-02T00:00:00,0.0,None,None,NaN,0.26000,1999-02-25T00:00:00,None,None,None,Акция,USD,1999-03-03T00:00:00,None,885042,4003
3,143995,G,G,RU,2024-04-12T17:51:06,ЕвроМуни,False,LU,NaN,None,1098.0,INTR,None,None,NaN,None,None,None,86681.0,Нижегородская Обл Прав,XS0080584740,XS0080584740,32-3-333,НижегородОбл-2005-еврооб,XS0080584740,8312,None,3.0,Фиксированный,None,1998-10-03T00:00:00,1999-04-03T00:00:00,180.0,8.75,437.5,4375000.0,10000.0,561.0,E,None,None,None,NaN,None,NaN,None,None,180.0,None,NaN,None,None,NaN,None,None,None,None,None,CPN,1999-04-03T00:00:00,1998-10-03T00:00:00,8.75,USD,NaN,NaN,None,437.5,USD,USD,NaN,0.00000,None,None,None,None,Облигация,None,None,None,885042,4004
4,498344,C,C,RU,2023-09-04T15:08:57,привилегированная акция,False,RU,3.0,None,1099.0,DVCA,None,None,NaN,Выплачивать деньгами,None,Выплачено,6495.0,ГАЗ,RU000A0ZYHF5,RU000A0ZYHF5,2-02-00029-A,ГАЗ-2-ап,RU000A0ZYHF5,4300,None,55.0,AM,1999-06-30T00:00:00,1998-01-01T00:00:00,1998-12-31T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1,1999-04-05T00:00:00,1999-03-01T00:00:00,None,NaN,None,NaN,None,None,365.0,None,NaN,None,None,NaN,None,None,None,None,None,DIV,1999-04-05T00:00:00,1998-01-01T00:00:00,NaN,RUB,12.00000,17442000.0,1999-07-05T00:00:

In [6]:
AccruedInterestOnDate = RuDataDF("AccruedInterestOnDate").df
AccruedInterestOnDate.head()

AccruedInterestOnDate started
AccruedInterestOnDate finished. AccruedInterestOnDate shape (101318, 14)


,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,7244,0,Calendar,None,2024-11-30T00:00:00,NaN,NaN,0.0,0.0,Typical,None,USD,USD,2024-11-30T00:00:00
1,7179,0,Calendar,None,2024-11-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2024-11-30T00:00:00
2,7060,0,Calendar,None,2024-11-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2024-11-30T00:00:00
3,7222,0,Calendar,None,2024-11-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,USD,USD,2024-11-30T00:00:00
4,7251,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '7251': Нет...,None,None,None


In [7]:
FloaterData = RuDataDF("FloaterData").df
FloaterData.head()

FloaterData started
FloaterData finished. FloaterData shape (1772, 10)


,fintoolId,beg_period,end_period,start_point,model_type,date_pub_n,date_pub_type,date_fix_n,date_fix_type,bases
0,11193,2000-12-01T00:00:00,2050-12-01T00:00:00,E,single,NaN,None,10.0,W,"[{'base_no': 1, 'calc_type': 'FIX', 'base_fint..."
1,30316,2015-03-18T00:00:00,2025-09-18T00:00:00,B,max,NaN,None,2.0,W,"[{'base_no': 4, 'calc_type': None, 'base_finto..."
2,30316,2025-09-18T00:00:00,2025-12-18T00:00:00,B,max,NaN,None,2.0,W,"[{'base_no': 5, 'calc_type': 'FIX', 'base_fint..."
3,36325,2009-08-27T00:00:00,2028-09-15T00:00:00,B,single,NaN,None,1.0,W,"[{'base_no': 1, 'calc_type': 'FIX', 'base_fint..."
4,59702,2013-12-11T00:00:00,2028-11-22T00:00:00,B,single,NaN,None,5.0,C,"[{'base_no': 2, 'calc_type': 'FIX', 'base_fint..."


#### Шаг 2. Предобработка таблиц

In [8]:
TransformedFintoolReferenceData = FintoolReferenceData[FintoolReferenceData['fintooltype']=='Облигация']\
.copy()[[
    'isincode', 'fintoolid', 'status', 'nickname', 'issuername', 'issuerinn', 'borrowername', 'borrowerinn',
    'fintooltype', 'facevalue', 'begdistdate', 'faceftname', 'nominaltype', 'coupontype', 'floatratename'
]]\
.drop_duplicates(subset=['isincode'])\
.rename(columns={
    'isincode': "isin",
    'fintoolid': "fintool",
    'status': "status",
    'nickname': "name",
    'issuername': "issuer",
    'issuerinn': "issuer_INN",
    'borrowername': "borrower",
    'borrowerinn': "borrower_INN",
    'fintooltype': "type",
    'facevalue': "face_value",
    'begdistdate': "issue_date",
    'faceftname': "currency",
    'nominaltype': "principal_type",
    'coupontype': "coupon_type",
    'floatratename': "rate"
})
TransformedFintoolReferenceData.set_index('isin', drop=True, inplace=True)
TransformedFintoolReferenceData['issue_date'] = pd.to_datetime(TransformedFintoolReferenceData['issue_date'])

Для обновления индексируемого номинала

In [9]:
INDEX_TYPES = ['Инфляционно-Индексируемый', 'Индексируемый', 'Инфляц-Индекс+Амортизация', 
               'Индексация по активу', 'Индексац.(актив)+ Аморт.', 'Инфляционно-Индексируемый']
index_types_isins = TransformedFintoolReferenceData[TransformedFintoolReferenceData['principal_type'].isin(INDEX_TYPES)].index

In [10]:
TransformedAccruedInterestOnDate = AccruedInterestOnDate.copy().dropna(subset=["currentFaceValue"])
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate\
.merge(FintoolReferenceData[['fintoolid', "isincode"]], how='left', left_on='fintoolId', right_on='fintoolid')\
.drop(columns=['fintoolid'])\
.rename(columns={
    "isincode": "isin", 
    'currentFaceValue': 'face_value', 
    'faceValueCurrency': 'currency'
})
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate[['isin', 'face_value', 'currency']].set_index('isin')

#Добавление загруженных данных в результирующий датафрейм
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate[~TransformedAccruedInterestOnDate.index.duplicated()]
TransformedAccruedInterestOnDateIndexed = TransformedAccruedInterestOnDate[
    TransformedAccruedInterestOnDate.index.isin(index_types_isins)
]
TransformedFintoolReferenceData.update(TransformedAccruedInterestOnDateIndexed)

In [11]:
TransformedFloaterData = FloaterData.copy()
TransformedFloaterData = pd\
.concat([TransformedFloaterData, pd.DataFrame([base[0] for base in FloaterData.bases])], axis=1)\
.drop(columns=['bases'])
TransformedFloaterData = TransformedFloaterData\
.merge(FintoolReferenceData[['fintoolid', "isincode"]], how='left', left_on='fintoolId', right_on='fintoolid')\
.drop(columns=['fintoolid'])\
.reset_index(drop=True)
TransformedFloaterData['beg_period'] = pd.to_datetime(TransformedFloaterData['beg_period'])
TransformedFloaterData['end_period'] = pd.to_datetime(TransformedFloaterData['end_period'])
TransformedFloaterData['premium_to_base'] /= 10000

In [12]:
TransformedCalendar = Calendar.copy()
TransformedCalendar.loc[TransformedCalendar['typeOperation'].isin(['O', 'N', 'P']), 'eventType'] = 'PUT'
TransformedCalendar['eventDate'] = pd.to_datetime(TransformedCalendar['eventDate'])
# TransformedCalendar['rateDate'] = pd.to_datetime(TransformedCalendar['rateDate'])
TransformedCalendar = TransformedCalendar[~TransformedCalendar['eventDate'].isna()]
TransformedCalendar.loc[~TransformedCalendar['revisedPay1Bond'].isna(), 'pay1Bond'] = \
TransformedCalendar.loc[~TransformedCalendar['revisedPay1Bond'].isna(), 'revisedPay1Bond']
TransformedCalendar['Currency_temp'] = TransformedCalendar['faceFTName']
TransformedCalendar.loc[~TransformedCalendar['currencyFTName'].isna(), 'Currency_temp'] = \
TransformedCalendar.loc[~TransformedCalendar['currencyFTName'].isna(), 'currencyFTName']
TransformedCalendar.loc[(~TransformedCalendar['paymentCurrencyFTName'].isna())&(~TransformedCalendar['revisedPay1Bond'].isna()), 'Currency_temp'] = \
TransformedCalendar.loc[(~TransformedCalendar['paymentCurrencyFTName'].isna())&(~TransformedCalendar['revisedPay1Bond'].isna()), 'paymentCurrencyFTName']
TransformedCalendar = TransformedCalendar[TransformedCalendar['eventType'].isin(['CPN', 'MTY', 'PUT'])] # если нужны оферты колл, нужно аккуратно подойти к выбору typeOperation
TransformedCalendar['value'] /= 100

In [13]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI
from time import sleep
from logger.Logger import Logger

key = 'CurrencyRate'
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
currencies_rudata = []
curr = TransformedCalendar[['Currency_temp', 'faceFTName']].drop_duplicates()
curr = curr[curr['Currency_temp'] != curr['faceFTName']].dropna().reset_index(drop=True)

def get_currency_rate(row):
    try:
        response = requests.post(url, json={'from': row['Currency_temp'], 'to': row['faceFTName'], 'date': REPORT_DATE_STR}, headers=RuDataRequest.headers)
    except Exception as e:
        sleep(10)
        Logger.exception(e)
        response = get_currency_rate(row)
    return response
    
for _, row in curr.iterrows():
    response = get_currency_rate(row)
    result: dict = response.json()
    result["from"] = row['Currency_temp']
    result["to"] = row['faceFTName']
    currencies_rudata.append(result)
    sleep(1)
CurrencyRate = pd.DataFrame(currencies_rudata)
CurrencyRate.rename(columns={"rate": "FX"}, inplace=True)
CurrencyRate.head()

CPU times: user 344 ms, sys: 22.1 ms, total: 366 ms
Wall time: 9.04 s


,FX,endOfDayRate,error,from,to
0,0.009282,0.009282,None,RUB,USD
1,0.008748,0.008748,None,RUB,EUR
2,1.268200,1.268200,None,GBP,USD
3,107.740900,107.740900,None,USD,RUB
4,0.007319,0.007319,None,RUB,GBP


In [3]:
currencies = pd.read_sql(
    """
    SELECT DISTINCT num, currency
    FROM currencies
    """
    , RuDataDF.engine).set_index("currency")["num"]

In [14]:
TransformedCalendar = TransformedCalendar.merge(CurrencyRate[["FX", "from", "to"]], how='left', left_on=['Currency_temp', 'faceFTName'], right_on=['from', 'to'])
TransformedCalendar['FX'] = TransformedCalendar['FX'].fillna(1)
TransformedCalendar['CF'] = TransformedCalendar['pay1Bond'] * TransformedCalendar['FX']
currencies = pd.read_sql(
    """
    SELECT DISTINCT num, currency
    FROM currencies
    """
    , RuDataDF.engine).set_index("currency")["num"]
TransformedCalendar['Currency'] = TransformedCalendar['faceFTName'].map(currencies).fillna(0).astype(np.int32)

In [15]:
TransformedCalendar = TransformedCalendar[[
    'isiNcode', 'finToolID', 'nickname', 'rateDate', 'eventDate', 'eventType', 
    'typeOperation', 'value', 'CF', 'faceFTName', 'note', 'eventTotalVol'
]].rename(columns={
    'isiNcode': "isin",
    'finToolID': "fintool",
    'nickname': "name",
    'rateDate': "rate_date", 
    'eventDate': "payment_date", 
    'eventType': "payment_type", 
    'typeOperation': 'operation_type',
    'value': "rate", 
    'CF': "payment_sum", 
    'faceFTName': "currency", 
    'note': "note",
    'eventTotalVol': 'event_total_vol'
})
TransformedCalendar.dropna(subset="isin", inplace=True)

#### Шаг 3. Корректировка потока

In [242]:
bonds = (
    TransformedCalendar
    .merge(
        TransformedFintoolReferenceData[["face_value", "principal_type", "coupon_type", "issue_date"]], 
        how='left', 
        left_on="isin", 
        right_index=True)\
    .rename(columns={
        "face_value": "nominal"
    })
)
bonds["nominal"] = bonds["nominal"].fillna(0)
bonds["rate"] = bonds["rate"].fillna(0)
bonds["payment_sum"] = bonds["payment_sum"].fillna(0)
bonds['event_total_vol'] = bonds['event_total_vol'].fillna(0)
bonds["name"] = bonds["name"].str.replace('[\n\t\r;,]', '.', regex=True)
bonds["note"] = bonds["note"].str.replace('[\n\t\r;,]', '.', regex=True)
bonds["payment_type_NUM"] = bonds["payment_type"].map({"CPN": 1, "MTY": 2, "PUT": 3})
bonds.sort_values(['isin', 'payment_date', 'payment_type_NUM'], inplace=True)

Обнуляем ставки по плавающим купонам, оцененных после отчетной даты

In [243]:
bonds.loc[(bonds['rate_date']>REPORT_DATE_STR) & (bonds['payment_type']=='CPN') & (bonds['operation_type']=='Плавающий'), ['rate', 'payment_sum']] = np.nan

Заполняем пропущенные ставки купона последним известным

In [244]:
bonds.loc[bonds['payment_type']=='CPN', 'rate'] = np.where(
    bonds.loc[bonds['payment_type']=='CPN', 'rate']<1e-6, 
    np.nan, 
    bonds.loc[bonds['payment_type']=='CPN', 'rate']
)
bonds.loc[bonds['payment_type']=='CPN', 'rate'] = (
    bonds.loc[bonds['payment_type']=='CPN', ['isin', 'rate']]
    .groupby('isin')['rate']
    .ffill()
)

Запоминаем ставку предыдущего купона

In [245]:
bonds['prev_cpn'] = (
    bonds
    .groupby("isin")["rate"]
    .shift(1)
).combine_first(bonds['rate'])

Заполняем разницу между датами выплат

In [246]:
bonds['prev_date'] = (
    bonds
    .groupby('isin')['payment_date']
    .shift(1)
)
bonds['days_diff'] = (bonds['payment_date'] - bonds['prev_date']).dt.days

Заполняем погашенный и непогашенный номинал

In [247]:
bonds["MTY"] = np.where(bonds.payment_type=="MTY", bonds['rate'], 0)
bonds["paid_nominal"] = (
    bonds
    .groupby(["isin"])["MTY"]
    .cumsum()
)
bonds["remain_nominal_no_shift"] = bonds["nominal"]*(1 - bonds["paid_nominal"])
bonds["remain_nominal"] = (
    bonds
    .groupby("isin")["remain_nominal_no_shift"]
    .shift(1)
    .fillna(bonds['nominal'])
)

если у нас perpetual, то есть нет даты погашения, добавляем в конец прогнозных потоков погашение

In [248]:
last_type = (
    bonds
    .groupby('isin')
    .last()
    .reset_index()
)
last_noMTY_isins = last_type[last_type['payment_type']=='CPN']['isin']
last_noMTY = last_type[last_type['isin'].isin(last_noMTY_isins)]
last_noMTY['payment_type'] = 'MTY'
last_noMTY['payment_type_NUM'] = 2
last_noMTY['payment_sum'] = (
    last_noMTY["isin"].map(
        bonds
        .groupby('isin')
        .last()["remain_nominal"]
        .to_dict()
    )
)
bonds = pd.concat(
    [bonds, last_noMTY], axis=0)\
.sort_values(['isin', 'payment_date', 'payment_type_NUM'])\
.reset_index(drop=True)

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_12379/3382992906.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_noMTY['payment_type'] = 'MTY'
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_12379/3382992906.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_noMTY['payment_type_NUM'] = 2
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_12379/3382992906.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

Корректируем поток для облигаций с индексируемым номиналом

In [249]:
indexed_bonds_isins = bonds[bonds['principal_type']=='Инфляционно-Индексируемый']['isin']
indexed_bonds = bonds[bonds['isin'].isin(indexed_bonds_isins)]
not_indexed_bonds = bonds[~bonds['isin'].isin(indexed_bonds_isins)]
cpn_pay_mask = indexed_bonds['payment_type']=='CPN'
mty_pay_mask = indexed_bonds['payment_type']=='MTY'
indexed_bonds.loc[cpn_pay_mask,'payment_sum'] = indexed_bonds.loc[cpn_pay_mask,'rate']*indexed_bonds.loc[cpn_pay_mask,'nominal']*indexed_bonds.loc[cpn_pay_mask,'days_diff']/365
indexed_bonds.loc[mty_pay_mask,'payment_sum'] = indexed_bonds.loc[mty_pay_mask,'remain_nominal']
bonds = pd.concat([indexed_bonds, not_indexed_bonds], axis=0).sort_values(['isin', 'payment_date', 'payment_type_NUM']).reset_index(drop=True)

оставляем только будущие потоки

In [250]:
bonds = bonds[bonds['payment_date']>REPORT_DATE]

обновляем погащение для индексируемого номинала

In [251]:
bonds.loc[bonds['isin'].isin(index_types_isins), "remain_nominal_no_shift"] = \
bonds.loc[bonds['isin'].isin(index_types_isins), "nominal"]*(1 - bonds.loc[bonds['isin'].isin(index_types_isins), "paid_nominal"])
bonds.loc[bonds['isin'].isin(index_types_isins), "remain_nominal"] = (
    bonds.loc[bonds['isin'].isin(index_types_isins), ['isin', "remain_nominal_no_shift"]]
    .groupby("isin")
    .shift(1)
    .fillna(bonds.loc[bonds['isin'].isin(index_types_isins), 'nominal'])
)
# индексируемый номинал уже обновлен
bonds.loc[bonds['isin'].isin(index_types_isins), "remain_nominal"] = \
bonds.loc[bonds['isin'].isin(index_types_isins), "nominal"]

обрезаем потоки по первую дату оферты

In [252]:
bonds['put'] = (
    bonds['payment_type']
    .eq('PUT')
    .groupby(bonds['isin'])
    .cumsum()
    .groupby(bonds['isin'])
    .shift(1)
    .fillna(0)
)
bonds = bonds[bonds['put']==0]

Удаляем погашенные потоки

In [253]:
bonds = bonds[bonds['remain_nominal']>1e-6]

Деление на флоатеры и обычные облигации

In [254]:
Bonds = bonds[~bonds['fintool'].isin(FloaterData["fintoolId"])]
floaters = bonds[bonds['fintool'].isin(FloaterData["fintoolId"])]

Флоатеры

Джойн лучше сделать в sql

In [255]:
import sqlite3

conn = sqlite3.connect(":memory:") 
TransformedFloaterData.to_sql("TransformedFloaterData", conn, index=False)
floaters.to_sql("floaters", conn, index=False)
Floaters = pd.read_sql_query("""
    SELECT 
        b.*
        , fl.beg_period
        , fl.end_period
        , fl.premium_to_base
        , fl.start_point
    FROM floaters b
    LEFT OUTER JOIN TransformedFloaterData fl ON b.fintool = fl.fintoolId AND b.payment_date >= fl.beg_period AND b.payment_date < fl.end_period
    """
    , conn
)
Floaters['payment_date'] = pd.to_datetime(Floaters['payment_date'])
Floaters.loc[Floaters['rate_date'].isna(), 'rate_date'] = date(1970, 1, 1)
Floaters['rate_date'] = pd.to_datetime(Floaters['rate_date'])

Запоминаем фиксированный купон

In [256]:
floaters_fix = Floaters[(Floaters['payment_type']=='CPN') & (Floaters['operation_type']!='Плавающий')]
floaters_fixed_dict = floaters_fix.groupby('isin').first().to_dict()['rate']
floaters_fix['rate'] = floaters_fix['isin'].map(floaters_fixed_dict)
floaters_float = Floaters[(Floaters['payment_type']=='CPN') & (Floaters['operation_type']=='Плавающий')].reset_index()

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_12379/559201119.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floaters_fix['rate'] = floaters_fix['isin'].map(floaters_fixed_dict)


Плавающие купоны

Если БЛИЖАЙШИЙ после расчетной даты будущий купон не пустой и дата rateDate больше расчетной даты, то делаем его пустым (с последующим заполнением предыдущим купоном)

In [257]:
floaters_float = Floaters[(Floaters['payment_type']=='CPN') & (Floaters['operation_type']=='Плавающий')]

In [258]:
floaters_float.loc[
            floaters_float['rate'].isna(), 'rate'
] = (
    floaters_float.loc[
            floaters_float['rate'].isna(), ['premium_to_base', 'prev_cpn']
    ].max(axis=1)
)

In [259]:
cpn_pay_nan_mask = (floaters_float['payment_sum']<1e-6) | (floaters_float['payment_sum'].isna())
floaters_float['days_diff'] = floaters_float['days_diff'].fillna(floaters_float.groupby('isin')['days_diff'].transform('mean'))
floaters_float.loc[cpn_pay_nan_mask, 'payment_sum'] = (
    floaters_float.loc[cpn_pay_nan_mask, 'rate']*
    floaters_float.loc[cpn_pay_nan_mask, 'remain_nominal']*
    floaters_float.loc[cpn_pay_nan_mask, 'days_diff']/365
)

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_12379/2330993031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floaters_float['days_diff'] = floaters_float['days_diff'].fillna(floaters_float.groupby('isin')['days_diff'].transform('mean'))


По той же формуле, где фиксированные купоны неизвестны

In [260]:
floaters_fix.loc[floaters_fix['payment_sum']<1e-6, 'payment_sum'] = (
    floaters_fix.loc[floaters_fix['payment_sum']<1e-6, 'rate']*
    floaters_fix.loc[floaters_fix['payment_sum']<1e-6, 'remain_nominal']*
    floaters_fix.loc[floaters_fix['payment_sum']<1e-6, 'days_diff']/365
)

Погашения

In [261]:
mty_floaters = Floaters[(Floaters['payment_type']!='CPN')]

Конкатенируем купоны и погашения

In [262]:
Floaters = (
    pd.concat(
        [floaters_fix, floaters_float, mty_floaters], axis=0
    ).sort_values(
        ['isin', 'payment_date', 'payment_type_NUM']
    )
)

#### Шаг 4. Объединение облигаций с постоянным и плавающим купоном в один датафрейм

In [263]:
Bonds["is_floater"] = False
Floaters["is_floater"] = True

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_12379/4080972275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bonds["is_floater"] = False


In [264]:
bonds = (
    pd.concat([Bonds, Floaters], axis=0)
    .sort_values(['isin', 'payment_date', 'payment_type_NUM'])
    .reset_index(drop=True)
)

In [265]:
bonds['id_payment'] = bonds.groupby(['isin']).cumcount() + 1
bonds["report_date"] = pd.to_datetime(REPORT_DATE)
bonds["payment_date"] = pd.to_datetime(bonds["payment_date"])

Мэппинг имен и кодов валют

In [266]:
bonds["currency"] = (
    bonds["currency"]
    .map(currencies)
    .fillna(0)
    .astype(np.int32)
)

Добавление времени выгрузки и выбор колонок

In [267]:
bonds = bonds[
    ["isin", "fintool", "name", "payment_date", "payment_type", "rate", "payment_sum", "currency", "report_date"]
]

In [268]:
bonds["export_timestamp"] = dt.now().strftime("%Y%m%d%H%M%S")
bonds.to_parquet(f'./data/Output/cash_flows.parquet', index=False)

##### Шаг 4.2. Write excel

In [293]:
from functions.send_email import main
main("cash_flows.parquet")